In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

In [ ]:
import pandas as pd
import scispacy
import spacy
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nlp = spacy.load("en_core_sci_sm")
meta = pd.read_csv("/kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv")

In [ ]:
vector_dict = {}
for sha, abstract in tqdm(meta[["sha","abstract"]].values):
    if isinstance(abstract, str):
        vector_dict[sha] = nlp(abstract).vector

In [ ]:
keys = list(vector_dict.keys())
values = list(vector_dict.values())

In [ ]:
cosine_sim_matrix = cosine_similarity(values, values)

In [ ]:
n_sim_articles = 5
input_sha = "e3b40cc8e0e137c416b4a2273a4dca94ae8178cc"


sha_index = keys.index(input_sha)
sim_indexes = np.argsort(cosine_sim_matrix[sha_index])[::-1][1:n_sim_articles+1]
sim_shas = [keys[i] for i in sim_indexes]
meta_info = meta[meta.sha.isin(sim_shas)]

In [ ]:
print("-------QUERY ABSTRACT-----")
print(meta[meta.sha == input_sha]["abstract"].values[0])

In [ ]:
print(f"----TOP {n_sim_articles} SIMILAR ABSTRACTS-----")
for abst in meta_info.abstract.values:
    print(abst)
    print("---------")

In [ ]:
n_return = 5
query_statement = "Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies"

In [ ]:
query_vector = nlp(query_statement).vector
cosine_sim_matrix_query = cosine_similarity(values, query_vector.reshape(1,-1))
query_sim_indexes = np.argsort(cosine_sim_matrix_query.reshape(1,-1)[0])[::-1][:n_return]
query_shas = [keys[i] for i in query_sim_indexes]
meta_info_query = meta[meta.sha.isin(query_shas)]

In [ ]:
print(f"----TOP {n_return} SIMILAR ABSTRACTS TO QUERY-----")
for abst in meta_info_query.abstract.values:
    print(abst)
    print("---------")